<a href="http://www.datascience-paris-saclay.fr">
<img src="img/logoUPSayPlusCDS_990.png" width="600px">
</a>

# Deep learning course for astro PhD students

##### Alexandre Boucaud (Paris Saclay CDS)
##### Marc Huertas-Company (LERMA)
##### Bertrand Rigaud (CCIN2P3)

1. [Introduction](#Introduction)
2. [Data](#Data)
3. [Workflow](#Workflow)
4. [Evaluation](#Evaluation)
5. [Local testing/exploration](#Local-testing)
6. [Submission](#Submitting-to-ramp.studio)

## Introduction

In astronomical images, the projection effects may cause two or more galaxies to overlap. When they are barely indistinguishable from one another, they are referred to as _blended_ and this can bias astrophysical estimators such as the morphology of galaxies or the shear (weak gravitational lensing distortion).  
As the sensitivity of imaging devices grows, a high fraction of galaxies appear _blended_ in the images, which is a known and important issue for current and upcoming galaxy surveys.  

In order not to discard such a wealth of information, it is key to develop methods to enable astronomers to alleviate such effect.
We can foresee some features that would help, in which machine learning could provide a solution:
- classify an image as containing isolated/blended objects  
  ___binary classification___
- count the number of blended sources in a blended image  
  ___regression / object detection___
- find the contours of each object  
  ___object detection/segmentation___
- ...

In this exercice, we will approach the third item, the detection of contours, but in a constrained way : the images will only contain **two galaxies** and the goal will be to find the **contours of the overlapping region** between both galaxies.

## Data



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!ls data

In [ ]:
X = np.load('data/data_test_mini.npy')
y = np.load('data/labels_test_mini.npy')

In [ ]:
def plot_data_basic(img, seg):
    if seg.ndim == 3:
        seg = seg.squeeze()
    if img.ndim == 3:
        img = img.squeeze()v

    fig_size = (8, 4)
    fig, ax = plt.subplots(nrows=1, ncols=2,
                           sharex=True, sharey=True, figsize=fig_size)
    ax[0].imshow(img, origin='lower')
    ax[0].axis('off')
    ax[0].set_title('blended galaxies')
    ax[1].imshow(seg, origin='lower',cmap='Greys_r')
    ax[1].axis('off')
    ax[1].set_title('segmap of overlaping region')

In [ ]:
index = 1
plot_data_basic(X[index], y[index])

## Workflow

The goal of the RAMP workflow is to help the participant focus on building optimized models to achieve a specific goal, by taking care of all other steps in the processing of the data. This can be summarized for this specific challenge with the following schematic

<img src="img/workflow.svg" width="150%">

The loading of the data, the actual training and testing and the scoring are taken care of behind the scenes, while the only input expected from participants is the ML model written as a _submission_ in a file called `object_detector.py`.

This file will be part of a submission `<submission_name>` that should be placed under the `submissions` directory such as
```
submissions/<submission_name>/object_detector.py
```

To get you started, we have already written two different models.
- `submissions/starting_kit/object_detector.py`
- `submissions/keras_fcnn/object_detector.py`

Inside `object_detector.py`, the workflow expects a Python class called `ObjectDetector` 
with two required methods implemented : `fit()` and `predict()`

```python
class ObjectDetector:
    def __init__(self): pass
    def fit(self): pass
    def predict(self): pass
``` 

### Dummy model - `starting-kit`

In [ ]:
# %load submissions/starting_kit/object_detector.py
import numpy as np

class ObjectDetector:
    def __init__(self):
        pass

    def fit(self, X, y):
        return self

    def predict(self, X):
        return np.zeros_like(X)

### Example deep net - `keras_fcnn`

In [ ]:
# %load submissions/keras_fcnn/object_detector.py
import numpy as np

from sklearn.utils import Bunch

from keras.models import Sequential
from keras.layers import Conv2D, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers.noise import GaussianNoise


class ObjectDetector(object):
    """Object detector.

    Parameters
    ----------
    batch_size : int, optional
        The batch size used during training. Set by default to 32 samples.

    epoch : int, optional
        The number of epoch for which the model will be trained. Set by default
        to 50 epochs.

    model_check_point : bool, optional
        Whether to create a callback for intermediate models.

    Attributes
    ----------
    model_ : object
        The DNN model.

    params_model_ : Bunch dictionary
        All hyper-parameters to build the DNN model.

    """

    def __init__(self, batch_size=32, epoch=10, model_check_point=True):
        self.model_, self.params_model_ = self._build_model()
        self.batch_size = batch_size
        self.epoch = epoch
        self.model_check_point = model_check_point

    def fit(self, X, y, pretrained=False):

        if pretrained:
            # for showcase load weights (this is not possible
            # for an actual submission)
            self.model_.load_weights(
                'submissions/keras_fcnn/fcnn_weights_best.h5')
            return

        # build the box encoder to later encode y to make usable in the model
        train_dataset = BatchGeneratorBuilder(X, y)
        train_generator, val_generator, n_train_samples, n_val_samples = \
            train_dataset.get_train_valid_generators(
                batch_size=self.batch_size)

        # create the callbacks to get during fitting
        callbacks = self._build_callbacks()

        # fit the model
        self.model_.fit_generator(
            generator=train_generator,
            steps_per_epoch=ceil(n_train_samples / self.batch_size),
            epochs=self.epoch,
            callbacks=callbacks,
            validation_data=val_generator,
            validation_steps=ceil(n_val_samples / self.batch_size))

    def predict(self, X):
        return self.model_.predict(np.expand_dims(X, -1))

    ###########################################################################
    # Setup model

    @staticmethod
    def _init_params_model():
        params_model = Bunch()

        # image and class parameters
        params_model.img_rows = 128
        params_model.img_cols = 128
        params_model.img_channels = 1

        # architecture params
        params_model.output_channels = 1            # size of the output in depth
        params_model.depth = 16                     # depth of all hidden layers
        params_model.n_layers = 6                   # number of layers before last
        params_model.conv_size0 = (3, 3)            # kernel size of first layer
        params_model.conv_size = (3, 3)             # kernel size of intermediate layers
        params_model.last_conv_size = (3, 3)        # kernel size of last layer
        params_model.activation = 'relu'            # activation between layers
        params_model.last_activation = 'sigmoid'    # final activation (sigmoid nice if binary objective)
        params_model.initialization = 'he_normal'   # weight initialization
        params_model.constraint = None              # kernel constraints (None, nonneg, unitnorm, maxnorm)
        params_model.dropout_rate = 0.0             # percentage of weights not updated (0 = no dropout)
        params_model.sigma_noise = 0.01             # random noise added before last layer (0 = no noise added)

        # optimizer parameters
        params_model.lr = 1e-4
        params_model.beta_1 = 0.9
        params_model.beta_2 = 0.999
        params_model.epsilon = 1e-08
        params_model.decay = 5e-05

        # loss parameters
        params_model.keras_loss = 'binary_crossentropy'

        # callbacks parameters
        params_model.early_stopping = True
        params_model.es_patience = 12
        params_model.es_min_delta = 0.001

        params_model.reduce_learning_rate = True
        params_model.lr_patience = 5
        params_model.lr_factor = 0.5
        params_model.lr_min_delta = 0.001
        params_model.lr_cooldown = 2

        return params_model

    def _build_model(self):

        # load the parameter for the SSD model
        params_model = self._init_params_model()

        model = fcnn_model(
            input_shape=(params_model.img_rows,
                         params_model.img_cols,
                         params_model.img_channels),
            output_channels=params_model.output_channels,
            depth=params_model.depth,
            n_layers=params_model.n_layers,
            conv_size0=params_model.conv_size0,
            conv_size=params_model.conv_size,
            last_conv_size=params_model.last_conv_size,
            activation=params_model.activation,
            last_activation=params_model.last_activation,
            dropout_rate=params_model.dropout_rate,
            sigma_noise=params_model.sigma_noise,
            initialization=params_model.initialization,
            constraint=params_model.constraint)

        optimizer = Adam(lr=params_model.lr)

        model.compile(optimizer=optimizer, loss=params_model.keras_loss)

        return model, params_model

    def _build_callbacks(self):
        callbacks = []

        if self.model_check_point:
            callbacks.append(
                ModelCheckpoint('./fcnn_weights_best.h5',
                                monitor='val_loss',
                                save_best_only=True,
                                save_weights_only=True,
                                period=1,
                                verbose=1))
        # add early stopping
        if self.params_model_.early_stopping:
            callbacks.append(
                EarlyStopping(monitor='val_loss',
                              min_delta=self.params_model_.es_min_delta,
                              patience=self.params_model_.es_patience,
                              verbose=1))

        # reduce learning-rate when reaching plateau
        if self.params_model_.reduce_learning_rate:
            callbacks.append(
                ReduceLROnPlateau(monitor='val_loss',
                                  factor=self.params_model_.lr_factor,
                                  patience=self.params_model_.lr_patience,
                                  cooldown=self.params_model_.lr_cooldown,
                                  # min_delta=self.params_model_.lr_min_delta,
                                  verbose=1))

        return callbacks


def fcnn_model(input_shape, output_channels, depth, n_layers,
               conv_size0, conv_size, last_conv_size,
               activation, last_activation,
               dropout_rate, sigma_noise,
               initialization, constraint):
    model = Sequential()
    model.add(Conv2D(depth, conv_size0,
                     input_shape=input_shape,
                     activation=activation,
                     padding='same',
                     name='conv0',
                     kernel_initializer=initialization,
                     kernel_constraint=constraint))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))

    for layer_n in range(1, n_layers):
        model.add(Conv2D(depth, conv_size,
                         activation=activation,
                         padding='same',
                         name="conv{}".format(layer_n),
                         kernel_initializer=initialization,
                         kernel_constraint=constraint))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    if sigma_noise > 0:
        model.add(GaussianNoise(sigma_noise))

    model.add(Conv2D(output_channels, last_conv_size,
                     activation=last_activation,
                     padding='same',
                     name='last',
                     kernel_initializer=initialization,
                     kernel_constraint=constraint))

    return model

## Evaluation

For image detection a classical metric is the ***Intersection over Union (IoU)*** also referred to as ***Jaccard index*** and defined as

$$ IoU(A, B) =  \dfrac{|A \cap B|}{|A \cup B|} $$

This metric is very sensitive to small shifts or area difference between truth and prediction, as can be seen on

<img src="img/iou_examples.png" width="70%">

Typically, a value of $ IoU > 0.5 $ is used to define a good detection.

An implementation of the IoU for a series of flatten segmentation images $\in [0, 1]$ is 

In [ ]:
def iou(y_true, y_pred):
    # epsilon value
    EPS = np.finfo(float).eps
    # convert array to boolean
    y_true_b = y_true.astype(bool)
    y_pred_b = y_pred.astype(bool)
    # compute binary intersection & union
    intersection = np.sum(y_true_b & y_pred_b, axis=-1)
    union = np.sum(y_true_b | y_pred_b, axis=-1)
    
    return (intersection + EPS) / (union + EPS)

This is the metric that will be used for this challenge.

## Local testing

The usual way to work with RAMP is to explore solutions, add feature transformations, select models, perhaps do some AutoML/hyperopt, etc., _locally_, and checking them with 

In [ ]:
!ramp_test_submission --quick-test

The script prints mean cross-validation scores 

```
Testing Astro PhD tutorial - galaxy deblending
Reading train and test files from ./data ...
Reading cv ...
Training ./submissions/starting_kit ...
CV fold 0
	score    IoU
	train  0.091|
	valid  0.029
	test   0.020
CV fold 1
	score    IoU
	train  0.045
	valid  0.121
	test   0.020
CV fold 2
	score    IoU
	train  0.075
	valid  0.061
	test   0.020
----------------------------
Mean CV scores
----------------------------
	score            IoU
	train  0.07 ± 0.0191
	valid  0.07 ± 0.0381
	test      0.02 ± 0.0
----------------------------
Bagged scores
----------------------------
	score    IoU
	valid  0.021
	test   0.023
```

It is <b><span style="color:red">important that you test your submission files before submitting them</span></b>. 

For this we provide a unit test. Note that the test runs on your files in [`submissions/starting_kit`](/tree/submissions/starting_kit), not on the classes defined in the cells of this notebook.

***Check list:***

- make sure `ramp-workflow` is installed locally (see [readme](README.md))
- make sure the data has been downloaded (see [readme](README.md))

- make sure a Python file `object_detector.py` is in the  `submissions/<your_submission>` folder

Finally, make sure there are no obvious code errors and the local processing goes through by running

In [ ]:
!ramp_test_submission --quick-test --submission <your_submission>

And if you want to perform the test on the full data set ***(only if downloaded)***

In [ ]:
!ramp_test_submission --submission <your_submission>

## Submitting to [ramp.studio](http://ramp.studio)

Once you found a good model, you can submit them to [ramp.studio](http://www.ramp.studio). First, if it is your first time using RAMP, [sign up](http://www.ramp.studio/sign_up), otherwise [log in](http://www.ramp.studio/login). 
Then find the event related to this challenge: [astrophd_tutorial](http://www.ramp.studio/events/astrophd_tutorial) and sign up for the event. 
Both signups are controled by RAMP administrators, so there **can be a delay between asking for signup and being able to submit**.

Once your signup request is accepted, you can go to your [sandbox](http://www.ramp.studio/events/astrophd_tutorial/sandbox) and copy-paste (or upload) [`object_detector.py`](/edit/submissions/starting_kit/object_detector.py). Save it, rename it, then submit it. The submission is trained and tested on our backend in the same way as `ramp_test_submission` does it locally. While your submission is waiting in the queue and being trained, you can find it in the "New submissions (pending training)" table in [my submissions](http://www.ramp.studio/events/astrophd_tutorial/my_submissions). Once it is trained, you get a mail, and your submission shows up on the [public leaderboard](http://www.ramp.studio/events/astrophd_tutorial/leaderboard). 
If there is an error (despite having tested your submission locally with `ramp_test_submission`), it will show up in the "Failed submissions" table in [my submissions](http://www.ramp.studio/events/astrophd_tutorial/my_submissions). You can click on the error to see part of the trace.

After submission, do not forget to give credits to the previous submissions you reused or integrated into your submission.

The data set we use at the backend is usually different from what you find in the starting kit, so the score may be different.

The official score in this RAMP (the first score column after "historical contributivity" on the [leaderboard](http://www.ramp.studio/events/astrophd_tutorial/leaderboard)) is the IoU.

### Contact

Don't hesitate to ask your questions on the CDS Slack channel [#phd_tutorial](https://cds-upsay.slack.com/messages/CA25J6FLL) so the answers are shared with everyone.

<footer style="float:right; color:#999;background:#fff;">
HAPPY CODING...
</footer>

---

In [25]:
from IPython.display import HTML
HTML(open("style/custom.css").read())